In [100]:
import numpy as np
import pandas as pd
import requests
import math
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.metrics import distance
# from sklearn.impute import SimpleImputer
# from tensorflow.keras.preprocessing.text import Tokenizer

In [15]:
df = pd.read_csv("../raw_data/steam_games.csv")
df.head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN


In [16]:
df['name']

0                                                     DOOM
1                            PLAYERUNKNOWN'S BATTLEGROUNDS
2                                               BATTLETECH
3                                                     DayZ
4                                               EVE Online
                               ...                        
40828    Rocksmith® 2014 Edition – Remastered – Sabaton...
40829    Rocksmith® 2014 Edition – Remastered – Stone T...
40830    Fantasy Grounds - Quests of Doom 4: A Midnight...
40831                         Mega Man X5 Sound Collection
40832                                     Stories In Stone
Name: name, Length: 40833, dtype: object

In [17]:
df.columns

Index(['url', 'types', 'name', 'desc_snippet', 'recent_reviews', 'all_reviews',
       'release_date', 'developer', 'publisher', 'popular_tags',
       'game_details', 'languages', 'achievements', 'genre',
       'game_description', 'mature_content', 'minimum_requirements',
       'recommended_requirements', 'original_price', 'discount_price'],
      dtype='object')

In [18]:
len(df.columns)

20

In [19]:
df.types.value_counts()

app       38021
bundle     2572
sub         238
Name: types, dtype: int64

In [20]:
df = df.drop(['recent_reviews', 'discount_price'], axis=1)
df = df[df['types']!='bundle']
df.head()

,url,types,name,desc_snippet,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free


In [21]:
df = df[df['types']!='sub']
df.shape

(38023, 18)

In [22]:
df['types'].isnull().sum()

2

In [23]:
df['types'] = df['types'].dropna() 
df.shape

(38023, 18)

In [24]:
df['types']=df['types'].astype(str)
df.shape

(38023, 18)

In [25]:
df=df[df['types']!='nan']

In [26]:
df.shape

(38021, 18)

In [27]:
my_df= df[['url','genre', 'game_description', 'mature_content',
       'minimum_requirements', 'recommended_requirements', 'original_price']].copy()
my_df

,url,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price
0,https://store.steampowered.com/app/379720/DOOM/,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99
1,https://store.steampowered.com/app/578080/PLAY...,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99
2,https://store.steampowered.com/app/637090/BATT...,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99
3,https://store.steampowered.com/app/221100/DayZ/,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99
4,https://store.steampowered.com/app/8500/EVE_On...,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free
...,...,...,...,...,...,...,...
40828,https://store.steampowered.com/app/899836/Rock...,"Casual,Simulation","About This Content Play ""Ghost Division"" by S...",NaN,"Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7, Win...",$2.99
40829,https://store.steampowered.com/app/899832/Rock...,"Casual,Simulation","About This Content Play ""Trippin’ on a Hole i...",NaN,"Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7, Win...",$2.99
40830,https://store.steampowered.com/app/906840/Fant...,"Indie,RPG,Strategy",About This Content Quests of Doom 4: A Midni...,NaN,"Minimum:,OS:,Windows 7x , 8x or 10x,Processor:...","Recommended:,OS:,Windows 7x , 8x or 10x,Proces...",$7.99
40831,https://store.steampowered.com/app/906635/Mega...,Action,About This Content Get equipped with the stun...,NaN,"Minimum:,OS:,WINDOWS® 7 (64bit),Processor:,Int...","Recommended:,OS:,WINDOWS®10 (64bit),Processor:...",$9.99


In [28]:
my_df['genre'].value_counts()

Action                                                               2236
Action,Indie                                                         2045
Casual,Indie                                                         1632
Action,Adventure,Indie                                               1503
Adventure,Indie                                                      1420
                                                                     ... 
Casual,Indie,RPG,Simulation,Sports,Strategy                             1
Action,Casual,Free to Play,Indie,Simulation,Strategy,Early Access       1
Action,Casual,Indie,RPG,Simulation,Strategy,Early Access                1
Action,Casual,Indie,Massively Multiplayer,Simulation,Strategy           1
Casual,Free to Play,Indie,Massively Multiplayer,Simulation,Sports       1
Name: genre, Length: 1185, dtype: int64

In [29]:
print(f"""number of genres: {my_df['genre'].unique().size}\n\
number of descriptions: {my_df['game_description'].unique().size}\n\
number of mature content: {my_df['mature_content'].unique().size}\n\
number of min req: {my_df['minimum_requirements'].unique().size}\n\
number of rec requ: {my_df['recommended_requirements'].unique().size}\n\
number of prices: {my_df['original_price'].unique().size}""")

number of genres: 1186
number of descriptions: 37497
number of mature content: 1268
number of min req: 14763
number of rec requ: 13837
number of prices: 630


In [30]:
my_df.isnull().sum().sort_values(ascending=False)/len(my_df)

mature_content              0.923805
minimum_requirements        0.445859
recommended_requirements    0.445701
original_price              0.079482
genre                       0.010415
game_description            0.002656
url                         0.000000
dtype: float64

In [31]:
# my_df[my_df['genre'].isna() ==True]

In [32]:
# my_df['genre']=my_df['genre'].astype(str)

In [33]:
# my_df['genre'] = my_df['genre'].dropna(axis=0)
# my_df

In [34]:
# my_df=my_df[my_df['genre']!='nan']

In [35]:
my_df.shape

(38021, 7)

In [36]:
# my_df['genre'].isna().sum()

In [37]:
len(my_df[my_df['game_description'].isna() ==True])

101

In [224]:
my_df[my_df['game_description'].isna() ==True]

,url,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price
1228,https://store.steampowered.com/app/3320/Insani...,Casual,NaN,NaN,NaN,NaN,$4.99
2797,https://store.steampowered.com/app/3300/Bejewe...,Casual,NaN,NaN,NaN,NaN,$4.99
2850,https://store.steampowered.com/app/3390/Feedin...,Casual,NaN,NaN,NaN,NaN,$4.99
3277,https://store.steampowered.com/app/3310/Chuzzl...,Casual,NaN,NaN,NaN,NaN,$4.99
3692,https://store.steampowered.com/app/3380/Dynomi...,Casual,NaN,NaN,NaN,NaN,$4.99
...,...,...,...,...,...,...,...
35820,https://store.steampowered.com/app/537210/Cand...,"Adventure,Indie",NaN,NaN,NaN,NaN,$4.99
37143,https://store.steampowered.com/app/261404/Epis...,"Casual,Indie",NaN,NaN,"Minimum:,OS:,Windows Vista / XP / 7,Processor:...","Recommended:,OS:,Windows Vista / XP / 7,Proces...",$0.99
37447,https://store.steampowered.com/app/549990/Coba...,NaN,NaN,NaN,NaN,NaN,NaN
38744,https://store.steampowered.com/app/439870/HITM...,Action,NaN,Mature Content Description The developers de...,NaN,NaN,NaN


In [39]:
my_df.shape

(38021, 7)

In [225]:
my_df2 =my_df.copy()

In [226]:
my_df2['game_description']=my_df2['game_description'].astype(str)

In [227]:
my_df2=my_df2[my_df2['game_description']!='nan']
my_df2.shape

(37920, 7)

In [230]:
len(my_df2)

37920

In [235]:
my_df2

,url,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price
0,https://store.steampowered.com/app/379720/DOOM/,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99
1,https://store.steampowered.com/app/578080/PLAY...,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99
2,https://store.steampowered.com/app/637090/BATT...,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99
3,https://store.steampowered.com/app/221100/DayZ/,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99
4,https://store.steampowered.com/app/8500/EVE_On...,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free
...,...,...,...,...,...,...,...
40828,https://store.steampowered.com/app/899836/Rock...,"Casual,Simulation","About This Content Play ""Ghost Division"" by S...",NaN,"Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7, Win...",$2.99
40829,https://store.steampowered.com/app/899832/Rock...,"Casual,Simulation","About This Content Play ""Trippin’ on a Hole i...",NaN,"Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7, Win...",$2.99
40830,https://store.steampowered.com/app/906840/Fant...,"Indie,RPG,Strategy",About This Content Quests of Doom 4: A Midni...,NaN,"Minimum:,OS:,Windows 7x , 8x or 10x,Processor:...","Recommended:,OS:,Windows 7x , 8x or 10x,Proces...",$7.99
40831,https://store.steampowered.com/app/906635/Mega...,Action,About This Content Get equipped with the stun...,NaN,"Minimum:,OS:,WINDOWS® 7 (64bit),Processor:,Int...","Recommended:,OS:,WINDOWS®10 (64bit),Processor:...",$9.99


In [237]:
my_df2[my_df2['game_description'].str.contains('About this content', case=False)]['url']

827      https://store.steampowered.com/app/441340/Elit...
1772     https://store.steampowered.com/app/57600/Tropi...
2105     https://store.steampowered.com/app/9070/DOOM_3...
2376     https://store.steampowered.com/app/201310/X3_A...
3478     https://store.steampowered.com/app/8650/RACE_0...
                               ...                        
40827    https://store.steampowered.com/app/899835/Rock...
40828    https://store.steampowered.com/app/899836/Rock...
40829    https://store.steampowered.com/app/899832/Rock...
40830    https://store.steampowered.com/app/906840/Fant...
40831    https://store.steampowered.com/app/906635/Mega...
Name: url, Length: 13111, dtype: object

In [217]:
my_df2['mature_content'].value_counts()

KeyError: 'mature_content'

In [218]:
my_df2['minimum_requirements'].value_counts()

Minimum:,OS:,Windows Vista, Windows 7, Windows 8,Processor:,2.66 GHz Intel Core2 Duo E6750 or 2.8 GHz AMD Athlon 64 X2 5600+,Memory:,2 GB RAM,Graphics:,256 MB DirectX 9 / NVIDIA® GeForce® 8600 GT or ATI Radeon™ HD 2600 XT,Hard Drive:,12 GB HD space,Sound:,DirectX 9.0c-compliant                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                742
Minimum:,OS:,Windows 7x , 8x or 10x,Processor:,1.6 GHz or higher processor,Memory:,1 GB RAM,Graphics:,Graphics card recommended,DirectX:,Versi

In [46]:
my_df2[my_df2.original_price.str.contains('Free')==True]

,url,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price
4,https://store.steampowered.com/app/8500/EVE_On...,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free
12,https://store.steampowered.com/app/323370/TERA/,"Action,Adventure,Free to Play,Massively Multip...",About This Game TERA is at the forefront of ...,Mature Content Description The developers de...,"Minimum:,OS:,Windows 7, 32-bit,Processor:,Inte...","Recommended:,OS:,Windows 7, 8, 8.1, 10, 64-bit...",Free to Play
18,https://store.steampowered.com/app/767560/War_...,"Action,Free to Play","About This Game War is raging, pilot! Are you...",NaN,"Minimum:,OS:,Windows 7,Processor:,2.5 GHz,Memo...","Recommended:,OS:,Windows 10,Processor:,3.2 GHz...",Free To Play
30,https://store.steampowered.com/app/372000/Tree...,"Free to Play,Massively Multiplayer,RPG","About This Game Tree of Savior (""TOS"") is an ...",NaN,"Minimum:,OS:,Windows 7 (64-bit),Processor:,Int...","Recommended:,OS:,Windows 7 (64-bit) or later,P...",Free to Play
36,https://store.steampowered.com/app/799960/Wiza...,"Free to Play,Massively Multiplayer,RPG",About This Game Embrace your inner Wizard and...,NaN,NaN,NaN,Free To Play
...,...,...,...,...,...,...,...
40361,https://store.steampowered.com/app/846360/DEEP...,"Action,Indie","About This Content Atmospheric, dynamic sound...",NaN,NaN,NaN,Free To Play
40669,https://store.steampowered.com/app/894440/Mast...,"Casual,Indie,Simulation",About This Content We present to you Belfast ...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",Free
40723,https://store.steampowered.com/app/876590/Meti...,Action,About This Content Original Score for Metis O...,NaN,"Minimum:,OS:,Any,Storage:,500 MB available space","Recommended:,OS:,Any,Storage:,500 MB available...",Free
40731,https://store.steampowered.com/app/876440/The_...,"Adventure,Indie",About This Content Take a look inside the Des...,NaN,NaN,NaN,Free


In [47]:
# free= my_df2[my_df2.original_price.str.contains('Free')==True]

In [48]:
# free['original_price'].value_counts()


In [49]:
# free = free[(free['original_price'].str.contains("free to play",case=False)==True)]
# free = free[(free['original_price'].str.contains("demo",case=False)==False)  
#            & (free['original_price'].str.contains("trial", case=False)==False)
#            & (free['original_price'].str.contains("guest", case=False)==False)
#            &(free['original_price'].str.contains("novel", case=False)==False)
#            &(free['original_price'].str.contains("prologue", case=False)==False)
#             &(free['original_price'].str.contains("mod", case=False)==False)]
# free['original_price'].value_counts()

In [50]:
# free[free['original_price'].str.contains("Geneshift Free Edition")==True]

In [51]:
# free[free['original_price'].str.contains("Experience Quasar Spectrum OS for Free")==True]

In [52]:
# free[free['original_price'].str.contains("Free test drives!")==True]

In [53]:
str(my_df2.iloc[7]['mature_content'])=='nan'

True

In [54]:
my_df2.iloc[7][['mature_content']].isna()

mature_content    True
Name: 8, dtype: bool

In [55]:
my_df2=my_df2.assign(mature_encoded=lambda x: x.mature_content.isna().replace((True,False),(0,1)))
my_df2.head()

,url,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,mature_encoded
0,https://store.steampowered.com/app/379720/DOOM/,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,0
1,https://store.steampowered.com/app/578080/PLAY...,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,1
2,https://store.steampowered.com/app/637090/BATT...,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,0
3,https://store.steampowered.com/app/221100/DayZ/,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,0
4,https://store.steampowered.com/app/8500/EVE_On...,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,0


In [56]:
my_df2= my_df2.drop(columns='mature_content')
my_df2.head()

,url,genre,game_description,minimum_requirements,recommended_requirements,original_price,mature_encoded
0,https://store.steampowered.com/app/379720/DOOM/,Action,"About This Game Developed by id software, the...","Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,0
1,https://store.steampowered.com/app/578080/PLAY...,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,1
2,https://store.steampowered.com/app/637090/BATT...,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,0
3,https://store.steampowered.com/app/221100/DayZ/,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,0
4,https://store.steampowered.com/app/8500/EVE_On...,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,0


In [57]:
def get_price(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find('div', class_='game_purchase_price').text.strip().split('\n')[1]
    except:
        return float('nan')

In [58]:
def get_price(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find('div', class_='game_purchase_price.price').text.strip('€')
    except AttributeError:
        return float('nan')

In [59]:
get_price('https://store.steampowered.com/app/907190/GRITS_Racing/')

nan

In [60]:
my_df2['genre'].value_counts()

Action                                                               2236
Action,Indie                                                         2045
Casual,Indie                                                         1632
Action,Adventure,Indie                                               1503
Adventure,Indie                                                      1420
                                                                     ... 
Casual,Indie,RPG,Simulation,Sports,Strategy                             1
Action,Casual,Free to Play,Indie,Simulation,Strategy,Early Access       1
Action,Casual,Indie,RPG,Simulation,Strategy,Early Access                1
Action,Casual,Indie,Massively Multiplayer,Simulation,Strategy           1
Casual,Free to Play,Indie,Massively Multiplayer,Simulation,Sports       1
Name: genre, Length: 1185, dtype: int64

In [61]:
### creating a loop of all the different tags, calculating the occurences to select the top used.
tags = {}
for index, row in df.iterrows():
    tags_list= str(row['popular_tags']).split(',')
    for tag in tags_list:
        if not tag in tags:
            tags[tag] = 1
        else:
            tags[tag] += 1
tags_df = pd.DataFrame(list(tags.items()),columns = ['tag','count'])
tags_df.sort_values('count', ascending =False) 

,tag,count
50,Indie,23718
2,Action,16112
42,Adventure,13378
75,Casual,12969
31,Simulation,9273
...,...,...
369,Snowboarding,3
364,BMX,3
362,ATV,3
202,Asymmetric VR,3


In [62]:
genre_dict = {}
for index, row in my_df2.iterrows():
    genre_list= str(row['genre']).split(',')
    for genre in genre_list:
        if not genre in genre_dict:
            genre_dict[genre] = 1
        else:
            genre_dict[genre] += 1
genre_df = pd.DataFrame(list(genre_dict.items()),columns = ['genre','count'])
genre_df.sort_values('count', ascending =False) 

,genre,count
6,Indie,22868
0,Action,15265
10,Casual,12010
1,Adventure,11871
8,Simulation,8711
3,Strategy,7975
5,RPG,6962
7,Early Access,2818
4,Free to Play,2620
11,Sports,1725


In [63]:
my_df2=my_df2.assign(price=lambda x: pd.to_numeric(x.original_price.str.strip('$'), errors='coerce').replace(np.nan, 0))
my_df2.head(20)

,url,genre,game_description,minimum_requirements,recommended_requirements,original_price,mature_encoded,price
0,https://store.steampowered.com/app/379720/DOOM/,Action,"About This Game Developed by id software, the...","Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,0,19.99
1,https://store.steampowered.com/app/578080/PLAY...,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,1,29.99
2,https://store.steampowered.com/app/637090/BATT...,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,0,39.99
3,https://store.steampowered.com/app/221100/DayZ/,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,0,44.99
4,https://store.steampowered.com/app/8500/EVE_On...,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,0,0.00
6,https://store.steampowered.com/app/601150/Devi...,Action,About This Game The Devil you know returns in...,"Minimum:,OS:,WINDOWS® 7, 8.1, 10 (64-BIT Requi...","Recommended:,OS:,WINDOWS® 7, 8.1, 10 (64-BIT R...",$59.99,1,59.99
7,https://store.steampowered.com/app/477160/Huma...,"Adventure,Indie","About This Game ***NEW ""DARK"" LEVEL AVAILABL...","Minimum:,OS:,Windows XP/Vista/7/8/8.1/10 x86 a...","Recommended:,OS:,Windows XP/Vista/7/8/8.1/10 x...",$14.99,0,14.99
8,https://store.steampowered.com/app/644930/They...,"Strategy,Early Access",About This Game They Are Billions is a strat...,"Minimum:,OS:,Windows 7, 8, 10 (32 and 64 bits)...","Recommended:,OS:,Windows 7, 8, 10 (64 bits),P...",$29.99,0,29.99
9,https://store.steampowered.com/app/774241/Warh...,"Action,Adventure,RPG","About This Game “Keep your eyes on this one,...","Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$49.99,1,49.99
10,https://store.steampowered.com/app/527230/For_...,"Adventure,Indie,RPG,Strategy","About This Game Into The Deep, a brand new a...","Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$19.99,0,19.99


In [64]:
my_df2['price'].dtypes

dtype('float64')

In [65]:
my_df2=my_df2.drop(columns='original_price')
my_df2.head()

,url,genre,game_description,minimum_requirements,recommended_requirements,mature_encoded,price
0,https://store.steampowered.com/app/379720/DOOM/,Action,"About This Game Developed by id software, the...","Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",0,19.99
1,https://store.steampowered.com/app/578080/PLAY...,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",1,29.99
2,https://store.steampowered.com/app/637090/BATT...,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",0,39.99
3,https://store.steampowered.com/app/221100/DayZ/,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",0,44.99
4,https://store.steampowered.com/app/8500/EVE_On...,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",0,0.00


In [66]:
# def get_os(req):
#     os_list=[]
#     os_s=['windows','macos','linux']
#     for os in os_s:
#         if req.str.contains(os,case=False):
#             os_list.append(os)
        
#     return os_list

# my_df2.assign(op_sys=lambda x: get_os(x.minimum_requiremts))


In [67]:
def operating_system(requirements):
    macos = [ 'macos','osx','os x','mac']
    linux = ['ubuntu','linux','red hat','fedora', 'mint','steamos', 'chrome os']
    existing = []
    if 'windows'in str(requirements).lower():
        existing.append('windows')
    for mac in macos:
        if mac in str(requirements).lower() and 'macOS' not in existing:
            existing.append('macOS')
    for lin in linux:
        if lin in str(requirements).lower() and 'Linux' not in existing:
            existing.append('Linux')
    return " ".join(existing)

# my_df2.assign(op_sys=lambda x: operating_system(x.minimum_requirements))

In [68]:

my_df2['op_sys']= my_df2['minimum_requirements'].apply(lambda x: operating_system(x))
my_df2

,url,genre,game_description,minimum_requirements,recommended_requirements,mature_encoded,price,op_sys
0,https://store.steampowered.com/app/379720/DOOM/,Action,"About This Game Developed by id software, the...","Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",0,19.99,windows
1,https://store.steampowered.com/app/578080/PLAY...,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",1,29.99,windows
2,https://store.steampowered.com/app/637090/BATT...,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",0,39.99,windows macOS Linux
3,https://store.steampowered.com/app/221100/DayZ/,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",0,44.99,windows
4,https://store.steampowered.com/app/8500/EVE_On...,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",0,0.00,windows macOS
...,...,...,...,...,...,...,...,...
40828,https://store.steampowered.com/app/899836/Rock...,"Casual,Simulation","About This Content Play ""Ghost Division"" by S...","Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7, Win...",0,2.99,windows
40829,https://store.steampowered.com/app/899832/Rock...,"Casual,Simulation","About This Content Play ""Trippin’ on a Hole i...","Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7, Win...",0,2.99,windows
40830,https://store.steampowered.com/app/906840/Fant...,"Indie,RPG,Strategy",About This Content Quests of Doom 4: A Midni...,"Minimum:,OS:,Windows 7x , 8x or 10x,Processor:...","Recommended:,OS:,Windows 7x , 8x or 10x,Proces...",0,7.99,windows
40831,https://store.steampowered.com/app/906635/Mega...,Action,About This Content Get equipped with the stun...,"Minimum:,OS:,WINDOWS® 7 (64bit),Processor:,Int...","Recommended:,OS:,WINDOWS®10 (64bit),Processor:...",0,9.99,windows


In [69]:
my_df2['op_sys'].value_counts()

                       18463
windows                15370
windows macOS Linux     1705
windows macOS           1385
windows Linux            720
macOS Linux              137
Linux                    130
macOS                    111
Name: op_sys, dtype: int64

In [196]:
my_df3= my_df2.drop(columns=['minimum_requirements', 'recommended_requirements']).copy()

In [71]:
my_df3.head()

,url,genre,game_description,mature_encoded,price,op_sys
0,https://store.steampowered.com/app/379720/DOOM/,Action,"About This Game Developed by id software, the...",0,19.99,windows
1,https://store.steampowered.com/app/578080/PLAY...,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,1,29.99,windows
2,https://store.steampowered.com/app/637090/BATT...,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,0,39.99,windows macOS Linux
3,https://store.steampowered.com/app/221100/DayZ/,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,0,44.99,windows
4,https://store.steampowered.com/app/8500/EVE_On...,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,0,0.00,windows macOS


In [72]:
def get_description(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find('div', class_='game_area_description').text.strip().split('\n')[1].strip('\t')
    except:
        return float('nan')

In [73]:
get_description('https://store.steampowered.com/app/3320/Insaniquarium_Deluxe/')

"Insaniquarium Deluxe is the craziest aquarium game ever - tend to your fish and keep them happy and they'll reward you with precious coins and jewels. Use the money you earn to buy upgrades for your tank or egg parts which hatch lots of different in-tank pets with special powers. These pets can help you feed your fish, collect coins, or even protect against the aliens that will invade your tank and wreak havoc."

In [74]:
def get_price(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find('div', class_='game_purchase_price').text.strip('\r').strip('\n').strip('\t').strip('€')
    except AttributeError:
        return float('nan')

In [75]:
get_price('https://store.steampowered.com/app/3320/Insaniquarium_Deluxe/')

'4,99'

In [76]:
get_price('https://store.steampowered.com/app/356190/Middleearth_Shadow_of_War/')

'Demo Middle-earth™: Shadow of War™'

In [77]:
my_df3[my_df3['genre'].isna() ==True]

,url,genre,game_description,mature_encoded,price,op_sys
211,https://store.steampowered.com/app/900883/The_...,NaN,About This Game The Elder Scrolls IV: Oblivio...,0,19.99,
528,https://store.steampowered.com/app/241930/Midd...,NaN,About This Game Fight through Mordor and unco...,0,19.99,Linux
1613,https://store.steampowered.com/app/71250/Sonic...,NaN,About This Game An ancient evil lurking withi...,0,7.99,windows
2254,https://store.steampowered.com/app/667720/Red_...,NaN,About This Game Set 50 years after the climac...,1,19.99,windows
2574,https://store.steampowered.com/app/758230/NEKO...,NaN,About This Video Nekopara is now turned into ...,0,29.99,windows macOS Linux
...,...,...,...,...,...,...
39860,https://store.steampowered.com/app/964550/Red_...,NaN,"About This Series In the distant future, two ...",0,1.99,windows macOS Linux
40133,https://store.steampowered.com/app/795330/Robo...,NaN,About This Series This tutorial is about how ...,0,3.99,windows macOS Linux
40234,https://store.steampowered.com/app/404110/Work...,NaN,About This Video In this tutorial I go throug...,0,9.99,windows macOS Linux
40367,https://store.steampowered.com/app/442310/Fore...,NaN,About This Video We will be creating a tiling...,0,49.99,windows macOS Linux


In [78]:
def get_genre(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    details = soup.find('div', class_='details_block').text.strip().split('\n')
    print(details)
#     details_dic = {}
        
#     for detail in details:
#         if detail.split(':')[0] != '':
#             details_dic[detail.split(':')[0]] = detail.split(':')[1]
        
#     details_dic['Price'] = float(soup.find('div', class_='price').text.strip('€').replace(',', '.'))
    try: 
        return soup.find('div', class_='game_purchase_price').text.strip('\r').strip('\n').strip('\t').strip('€')
    except AttributeError:
        return float('nan')
    return details[1].split(':')[1]

In [79]:
get_genre('https://store.steampowered.com/app/356190/Middleearth_Shadow_of_War/')

['Title: Middle-earth™: Shadow of War™', 'Genre: Action, Adventure, RPG', '', 'Developer:', 'Monolith Productions', '', '', 'Publisher:', 'WB Games', '', 'Release Date: 9 Oct, 2017']


'Demo Middle-earth™: Shadow of War™'

In [255]:
get_genre('https://store.steampowered.com/app/241930/Middleearth_Shadow_of_Mordor/')

['Title: Middle-earth™: Shadow of Mordor™', '', 'Developer:', 'Monolith Productions', '', '', 'Publisher:', 'Warner Bros. Interactive Entertainment, Warner Bros. Games', '', 'Release Date: 30 Sep, 2014']


IndexError: list index out of range

In [257]:
get_genre('https://store.steampowered.com/app/71250/Sonic_Adventure_DX/')

['Title: Sonic Adventure DX', '', 'Developer:', 'SEGA', '', '', 'Publisher:', 'SEGA', '', '', 'Franchise:', 'Sonic the Hedgehog', '', 'Release Date: 4 Mar, 2011']


IndexError: list index out of range

In [259]:
get_genre('https://store.steampowered.com/app/900883/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition_Deluxe/')

['Title: The Elder Scrolls IV: Oblivion® Game of the Year Edition Deluxe', '', 'Developer:', 'Bethesda Game Studios®', '', '', 'Publisher:', 'Bethesda Softworks', '', 'Release Date: 16 Jun, 2009']


IndexError: list index out of range

In [80]:
def get_price(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find('div', class_='game_purchase_price').text.strip('\r').strip('\n').strip('\t').strip('€')
    except AttributeError:
        return float('nan')

In [81]:
my_df['game_description'][0]

' About This Game Developed by id software, the studio that pioneered the first-person shooter genre and created multiplayer Deathmatch, DOOM returns as a brutally fun and challenging modern-day shooter experience. Relentless demons, impossibly destructive guns, and fast, fluid movement provide the foundation for intense, first-person combat – whether you’re obliterating demon hordes through the depths of Hell in the single-player campaign, or competing against your friends in numerous multiplayer modes. Expand your gameplay experience using DOOM SnapMap game editor to easily create, play, and share your content with the world. STORY: You’ve come here for a reason. The Union Aerospace Corporation’s massive research facility on Mars is overwhelmed by fierce and powerful demons, and only one person stands between their world and ours.  As the lone DOOM Marine, you’ve been activated to do one thing – kill them all. KEY FEATURES: A Relentless Campaign There is no taking cover or stopping t

In [197]:
def clean (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return lemmatized

In [83]:
clean(my_df['game_description'][0])

['game',
 'developed',
 'id',
 'software',
 'studio',
 'pioneered',
 'first',
 'person',
 'shooter',
 'genre',
 'created',
 'multiplayer',
 'deathmatch',
 'doom',
 'return',
 'brutally',
 'fun',
 'challenging',
 'modern',
 'day',
 'shooter',
 'experience',
 'relentless',
 'demon',
 'impossibly',
 'destructive',
 'gun',
 'fast',
 'fluid',
 'movement',
 'provide',
 'foundation',
 'intense',
 'first',
 'person',
 'combat',
 'whether',
 'obliterating',
 'demon',
 'horde',
 'depth',
 'hell',
 'single',
 'player',
 'campaign',
 'competing',
 'friend',
 'numerous',
 'multiplayer',
 'mode',
 'expand',
 'gameplay',
 'experience',
 'using',
 'doom',
 'snapmap',
 'game',
 'editor',
 'easily',
 'create',
 'play',
 'share',
 'content',
 'world',
 'story',
 'come',
 'reason',
 'union',
 'aerospace',
 'corporation',
 'massive',
 'research',
 'facility',
 'mar',
 'overwhelmed',
 'fierce',
 'powerful',
 'demon',
 'one',
 'person',
 'stand',
 'world',
 'lone',
 'doom',
 'marine',
 'activated',
 'one',
 

In [198]:
my_df3['clean_description'] = my_df3.game_description.astype('str').apply(clean)
# my_df3['clean_description'] = my_df3['clean_description'].astype('str')

In [199]:
my_df3.head()

,url,genre,game_description,mature_encoded,price,op_sys,clean_description
0,https://store.steampowered.com/app/379720/DOOM/,Action,"About This Game Developed by id software, the...",0,19.99,windows,"[game, developed, id, software, studio, pionee..."
1,https://store.steampowered.com/app/578080/PLAY...,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,1,29.99,windows,"[game, playerunknown, battleground, battle, ro..."
2,https://store.steampowered.com/app/637090/BATT...,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,0,39.99,windows macOS Linux,"[game, original, battletech, mechwarrior, crea..."
3,https://store.steampowered.com/app/221100/DayZ/,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,0,44.99,windows,"[game, post, soviet, country, chernarus, struc..."
4,https://store.steampowered.com/app/8500/EVE_On...,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,0,0.00,windows macOS,[game]


In [86]:
my_df3=my_df3.drop(columns='game_description')


In [87]:
my_df3.head()

,url,genre,mature_encoded,price,op_sys,clean_description
0,https://store.steampowered.com/app/379720/DOOM/,Action,0,19.99,windows,"[game, developed, id, software, studio, pionee..."
1,https://store.steampowered.com/app/578080/PLAY...,"Action,Adventure,Massively Multiplayer",1,29.99,windows,"[game, playerunknown, battleground, battle, ro..."
2,https://store.steampowered.com/app/637090/BATT...,"Action,Adventure,Strategy",0,39.99,windows macOS Linux,"[game, original, battletech, mechwarrior, crea..."
3,https://store.steampowered.com/app/221100/DayZ/,"Action,Adventure,Massively Multiplayer",0,44.99,windows,"[game, post, soviet, country, chernarus, struc..."
4,https://store.steampowered.com/app/8500/EVE_On...,"Action,Free to Play,Massively Multiplayer,RPG,...",0,0.00,windows macOS,[game]


In [295]:
my_df3.loc[4]['url']

'https://store.steampowered.com/app/8500/EVE_Online/'

In [200]:
name_df = df[['url','name']]
name_df.head()

,url,name
0,https://store.steampowered.com/app/379720/DOOM/,DOOM
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH
3,https://store.steampowered.com/app/221100/DayZ/,DayZ
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online


In [201]:
my_df3=my_df3.merge(name_df, on='url', how='inner')

In [202]:
def preprocessing(text):
    for punctuation in string.punctuation: 
        text = text.replace(punctuation, ' ').lower() 
    return text

In [203]:
my_df3['no_punct_name'] = my_df3['name'].apply(lambda x: preprocessing(str(x)))
my_df3.head()

,url,genre,game_description,mature_encoded,price,op_sys,clean_description,name,no_punct_name
0,https://store.steampowered.com/app/379720/DOOM/,Action,"About This Game Developed by id software, the...",0,19.99,windows,"[game, developed, id, software, studio, pionee...",DOOM,doom
1,https://store.steampowered.com/app/578080/PLAY...,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,1,29.99,windows,"[game, playerunknown, battleground, battle, ro...",PLAYERUNKNOWN'S BATTLEGROUNDS,playerunknown s battlegrounds
2,https://store.steampowered.com/app/637090/BATT...,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,0,39.99,windows macOS Linux,"[game, original, battletech, mechwarrior, crea...",BATTLETECH,battletech
3,https://store.steampowered.com/app/221100/DayZ/,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,0,44.99,windows,"[game, post, soviet, country, chernarus, struc...",DayZ,dayz
4,https://store.steampowered.com/app/8500/EVE_On...,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,0,0.00,windows macOS,[game],EVE Online,eve online


In [204]:
len(my_df3['no_punct_name'].unique())

37940

In [110]:
distance.edit_distance_align('doom ddfg', 'Doooumm4  dgd')

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (5, 7),
 (5, 8),
 (5, 9),
 (6, 10),
 (6, 11),
 (7, 11),
 (8, 12),
 (9, 12),
 (9, 13)]

In [205]:
def lev_dist(string1, string2):
    if distance.edit_distance(string1,string2) <= 4:
        return 'match'
    else:
        return 'no match'

In [206]:
user_df= pd.read_csv('../raw_data/steam-200k.csv',usecols=[0,1,2,3],names=['userid','game','behavior','hoursplayed'])
user_df.head()

,userid,game,behavior,hoursplayed
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0


In [207]:
game_list = list(user_df['game'].unique())
game_list

['The Elder Scrolls V Skyrim',
 'Fallout 4',
 'Spore',
 'Fallout New Vegas',
 'Left 4 Dead 2',
 'HuniePop',
 'Path of Exile',
 'Poly Bridge',
 'Left 4 Dead',
 'Team Fortress 2',
 'Tomb Raider',
 'The Banner Saga',
 'Dead Island Epidemic',
 'BioShock Infinite',
 'Dragon Age Origins - Ultimate Edition',
 'Fallout 3 - Game of the Year Edition',
 'SEGA Genesis & Mega Drive Classics',
 'Grand Theft Auto IV',
 'Realm of the Mad God',
 'Marvel Heroes 2015',
 'Eldevin',
 'Dota 2',
 'BioShock',
 'Robocraft',
 "Garry's Mod",
 'Jazzpunk',
 'Alan Wake',
 'BioShock 2',
 'Fallen Earth',
 "Fallout New Vegas Courier's Stash",
 'Fallout New Vegas Dead Money',
 'Fallout New Vegas Honest Hearts',
 'Grand Theft Auto Episodes from Liberty City',
 'Hitman Absolution',
 'HuniePop Official Digital Art Collection',
 'HuniePop Original Soundtrack',
 'The Banner Saga - Mod Content',
 'The Elder Scrolls V Skyrim - Dawnguard',
 'The Elder Scrolls V Skyrim - Dragonborn',
 'The Elder Scrolls V Skyrim - Hearthfire',


In [208]:
game_list_2= list(my_df3['no_punct_name'].unique())

In [210]:
for game in game_list[:300]:
    for game2 in game_list_2[:300]:
        if lev_dist(game,game2) == 'match':
            print(f"{game} -> {game2}")


Fallout 4 -> fallout 4 vr
Spore -> doom
Spore -> tera
Spore -> portal
Spore -> forts
Spore -> spore™
Spore -> gorn
Spore -> eco
Spore -> trove
Spore -> uno
Spore -> prey
Fallout New Vegas -> fallout  new vegas
Left 4 Dead 2 -> left 4 dead 2
Left 4 Dead -> left 4 dead 2
BioShock Infinite -> bioshock infinite
Realm of the Mad God -> realm of the mad god
Dota 2 -> portal
Dota 2 -> xcom® 2
Dota 2 -> portal 2
Dota 2 -> rage 2
Garry's Mod -> garry s mod
7 Days to Die -> 7 days to die
Killing Floor -> killing floor 2
Company of Heroes -> company of heroes 2
Undertale -> undertale
Company of Heroes 2 -> company of heroes 2
Sanctum -> scum
Far Cry 3 -> far cry® 5
Far Cry 4 -> far cry® 5
Just Cause 2 -> just cause™ 3
Portal 2 -> portal
Portal 2 -> portal 2
Tropico 4 -> tropico 6
DayZ -> doom
DayZ -> dayz
DayZ -> tera
DayZ -> gorn
DayZ -> eco
DayZ -> dusk
DayZ -> scum
DayZ -> uno
DayZ -> prey
DayZ -> raft
Monaco -> eco
Monaco -> uno
Ori and the Blind Forest -> ori and the blind forest
Sniper Elit

In [211]:
user_df['name']=user_df['game']

In [143]:
user_test= pd.DataFrame(user_df['name'].unique(),columns=['name'])
user_test.head()

,name
0,The Elder Scrolls V Skyrim
1,Fallout 4
2,Spore
3,Fallout New Vegas
4,Left 4 Dead 2


In [144]:
len(my_df3)

38021

In [145]:
test_df=my_df3.merge(user_test, on='name')
test_df.head()

,url,genre,mature_encoded,price,op_sys,clean_description,name,no_punct_name
0,https://store.steampowered.com/app/221100/DayZ/,"Action,Adventure,Massively Multiplayer",0,44.99,windows,"[game, post, soviet, country, chernarus, struc...",DayZ,dayz
1,https://store.steampowered.com/app/8500/EVE_On...,"Action,Free to Play,Massively Multiplayer,RPG,...",0,0.00,windows macOS,[game],EVE Online,eve online
2,https://store.steampowered.com/app/323370/TERA/,"Action,Adventure,Free to Play,Massively Multip...",1,0.00,windows,"[game, tera, forefront, new, breed, mmo, true,...",TERA,tera
3,https://store.steampowered.com/app/253250/Ston...,"Indie,Simulation,Strategy",0,19.99,windows macOS,"[game, stonehearth, pioneer, living, world, fu...",Stonehearth,stonehearth
4,https://store.steampowered.com/app/12210/Grand...,"Action,Adventure",0,19.99,,"[game, please, note, microsoft, longer, suppor...",Grand Theft Auto IV,grand theft auto iv


In [146]:
len(test_df)

1982

In [195]:
def re_clean(text):
    non_imp_words=['game','please','genre','whether']
    for word in non_imp_words:
        temp=text.copy()
        while word in temp:
            temp.remove(word)
    return temp

In [193]:
# my_df3=my_df3.assign(cleaner_desc=lambda x: re_clean(x.clean_description))
my_df3['cleaner_desc']=my_df3['clean_description'].apply(re_clean)

In [194]:
my_df3.head()

,url,genre,mature_encoded,price,op_sys,clean_description,name,no_punct_name,cleaner_desc
0,https://store.steampowered.com/app/379720/DOOM/,Action,0,19.99,windows,"[pioneered, first, person, shooter, created, m...",DOOM,doom,"[first, person, shooter, created, multiplayer,..."
1,https://store.steampowered.com/app/578080/PLAY...,"Action,Adventure,Massively Multiplayer",1,29.99,windows,"[playerunknown, battleground, battle, royale, ...",PLAYERUNKNOWN'S BATTLEGROUNDS,playerunknown s battlegrounds,"[playerunknown, battleground, battle, royale, ..."
2,https://store.steampowered.com/app/637090/BATT...,"Action,Adventure,Strategy",0,39.99,windows macOS Linux,"[original, battletech, mechwarrior, creator, j...",BATTLETECH,battletech,"[original, battletech, mechwarrior, creator, j..."
3,https://store.steampowered.com/app/221100/DayZ/,"Action,Adventure,Massively Multiplayer",0,44.99,windows,"[post, soviet, chernarus, struck, unknown, vir...",DayZ,dayz,"[post, soviet, chernarus, struck, unknown, vir..."
4,https://store.steampowered.com/app/8500/EVE_On...,"Action,Free to Play,Massively Multiplayer,RPG,...",0,0.00,windows macOS,[],EVE Online,eve online,[]


In [174]:
re_clean(my_df3.loc[0]['clean_description'])

['developed',
 'id',
 'software',
 'studio',
 'pioneered',
 'first',
 'person',
 'shooter',
 'genre',
 'created',
 'multiplayer',
 'deathmatch',
 'doom',
 'return',
 'brutally',
 'fun',
 'challenging',
 'modern',
 'day',
 'shooter',
 'experience',
 'relentless',
 'demon',
 'impossibly',
 'destructive',
 'gun',
 'fast',
 'fluid',
 'movement',
 'provide',
 'foundation',
 'intense',
 'first',
 'person',
 'combat',
 'whether',
 'obliterating',
 'demon',
 'horde',
 'depth',
 'hell',
 'single',
 'player',
 'campaign',
 'competing',
 'friend',
 'numerous',
 'multiplayer',
 'mode',
 'expand',
 'gameplay',
 'experience',
 'using',
 'doom',
 'snapmap',
 'editor',
 'easily',
 'create',
 'play',
 'share',
 'content',
 'world',
 'story',
 'come',
 'reason',
 'union',
 'aerospace',
 'corporation',
 'massive',
 'research',
 'facility',
 'mar',
 'overwhelmed',
 'fierce',
 'powerful',
 'demon',
 'one',
 'person',
 'stand',
 'world',
 'lone',
 'doom',
 'marine',
 'activated',
 'one',
 'thing',
 'kill',


In [ ]:
def get_price(url):
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    try: 
        return soup.find('div', class_='game_purchase_price').text.strip('\r').strip('\n').strip('\t').strip('€')
    except AttributeError:
        return float('nan')

In [2]:
simple = list(range(20,90))
simple

[20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89]

In [3]:
for params in simple:
    print(params.index)

AttributeError: 'int' object has no attribute 'index'